# Processing of Slocum Glider-AD2CP Data: RRU36 2023

jgradone@marine.rutgers.edu     03/10/2022    Initial <br>
jgradone@marine.rutgers.edu     06/16/2022    Update for pre-processing <br>
engdahl@marine.rutgers.edu      01/19/2024    Modify Notebook 2, 3 & 4 with new example using nbronikowski@mun.ca functions


**This Jupyter Notebook is intended to:**<br>
1) Read glider data frome ERDDAP <br>
2) Read in AD2CP data  <br>
3) Pre-porcess and Least squares linear inversion on ADCP velocities referenced to true ocean velocity through a depth averaged urrent constraint <br>
4) Save output from each segment<br>

*Details/comments on what the functions are actually doing in the source code*

In [1]:
# Imports
import scipy.interpolate as interp
from scipy.sparse.linalg import lsqr
import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import netCDF4 as nc
import math
import datetime
import xarray as xr
import matplotlib.dates as mdates
import dask.array as da
from erddapy import ERDDAP
from netCDF4 import Dataset
import gsw
import cmocean.cm as cmo
import sys
from datetime import datetime


## To import functions from Slocum-AD2CP GitHub repository, make this path the path to where the repo exists locally
sys.path.insert(0,'../src/analysis/')
sys.path.insert(0,'../src/data/')
from make_dataset import correct_sound_speed, beam_true_depth, cell_vert, binmap_adcp, beam2enu, inversion, qaqc_pre_coord_transform, qaqc_post_coord_transform
from analysis import get_erddap_dataset



import importlib
import NicolaiFunctions
importlib.reload(NicolaiFunctions)
from NicolaiFunctions import calcAHRS as calcAHRS_NI
from NicolaiFunctions import beam2enu as beam2enu_NI
from NicolaiFunctions import correct_ad2cp_heading
from NicolaiFunctions import inversion as inversion_NI

## Step 1: Load Glider Data

In [2]:
ds_id = 'ru36-20230525T1748-trajectory-raw-delayed'


## Load flight data
variables = ['depth', 'latitude', 'longitude', 'time', 'sci_water_temp', 'sci_water_cond','source_file', 'm_water_vx', 'm_water_vy', 'm_heading']
gdf = get_erddap_dataset(ds_id, server='http://slocum-data.marine.rutgers.edu/erddap', variables = variables, filetype='dataframe')
gdf.columns = variables

## Great way to find start and end times!!
start_times = gdf.groupby('source_file').first().time.values
end_times   = gdf.groupby('source_file').last().time.values
## Remove time zone for slicing ad2cp times
start_times2 = pd.to_datetime(start_times).tz_localize(None)
end_times2 = pd.to_datetime(end_times).tz_localize(None)

gdf

,depth,latitude,longitude,time,sci_water_temp,sci_water_cond,source_file,m_water_vx,m_water_vy,m_heading
0,0.000000,38.026363,-63.452682,2023-05-25T17:48:02.52Z,0.0000,0.00000,ru36-2023-144-1-0-dbd(01180000),NaN,NaN,NaN
1,0.000000,38.026363,-63.452682,2023-05-25T17:48:11.39Z,0.0000,0.00000,ru36-2023-144-1-0-dbd(01180000),NaN,NaN,NaN
2,NaN,38.026363,-63.452682,2023-05-25T17:48:12.28Z,NaN,NaN,ru36-2023-144-1-0-dbd(01180000),0.0,0.0,6.2221
3,NaN,38.026363,-63.452682,2023-05-25T17:48:12.39Z,NaN,NaN,ru36-2023-144-1-0-dbd(01180000),NaN,NaN,NaN
4,NaN,38.026363,-63.452682,2023-05-25T17:48:13.39Z,NaN,NaN,ru36-2023-144-1-0-dbd(01180000),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1686329,-0.071364,37.708178,-63.127247,2023-06-10T09:27:12.48Z,20.8637,5.08320,ru36-2023-158-0-9-dbd(01240009),NaN,NaN,NaN
1686330,-0.073349,37.708178,-63.127247,2023-06-10T09:27:13.48Z,20.8640,5.08318,ru36-2023-158-0-9-dbd(01240009),NaN,NaN,NaN
1686331,-0.047047,37.708178,-63.127247,2023-06-10T09:27:14.48Z,20.8659,5.08337,ru36-2023-158-0-9-dbd(01240009),NaN,NaN,NaN
1686332,NaN,37.708178,-63.127247,2023-06-10T09:27:15.48Z,NaN,NaN,ru36-2023-158-0-9-dbd(01240009),NaN,NaN,NaN


## Step 2: Load in AD2CP data frame

In [3]:
path='../Passengers/AD2CP_data_leg2/pass2_0000_netcdf_100000kb_partition_ncs_no_header/'
files = np.sort(glob.glob(path+'*.nc'))

tot_ad2cp = xr.open_mfdataset(files,group='Data/Burst/', concat_dim="time", combine="nested")
## Because files are not necessarily read in time order with above line
tot_ad2cp = tot_ad2cp.sortby('time')
## I think this is a documented bug in the xarray open_mfdataset function for grouped NetCDFs
config = xr.open_dataset(files[0],group='Config')
## So, just assigning the global attributes from the first file to the combined file
tot_ad2cp = tot_ad2cp.assign_attrs(config.attrs)
tot_ad2cp = tot_ad2cp.rename({'Velocity Range':'VelocityRange','Correlation Range':'CorrelationRange','Amplitude Range':'AmplitudeRange'})
# 2 House-keeping steps
# 1) Roll shifted 180 for some reason
tot_ad2cp['Roll'] = tot_ad2cp['Roll'] 
# 2) Surface depth is 10 meters. Needed for at least for the 2020 and both 2021 deployments, per conversations with Sven from Nortek.
tot_ad2cp['Pressure'] = tot_ad2cp['Pressure']-10
# Put time on x-dimension
tot_ad2cp = tot_ad2cp.transpose()

# specific to this deployment because it was recovered 6/14 , need to subset time
tot_ad2cp=tot_ad2cp.sel(time=slice('2023-05-25', '2023-06-14'))
tot_ad2cp

C:\Users\JuliaEngdahl\miniconda3\envs\rucool\lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:                    (time: 4935810, VelocityRange: 15,
                                CorrelationRange: 15, AmplitudeRange: 15,
                                Physicalbeam_dim: 4)
Coordinates:
  * time                       (time) datetime64[ns] 2023-05-25T17:48:31.0320...
  * VelocityRange              (VelocityRange) float32 2.2 4.2 6.2 ... 28.2 30.2
  * CorrelationRange           (CorrelationRange) float32 2.2 4.2 ... 28.2 30.2
  * AmplitudeRange             (AmplitudeRange) float32 2.2 4.2 ... 28.2 30.2
Dimensions without coordinates: Physicalbeam_dim
Data variables: (12/43)
    VelocityBeam1              (VelocityRange, time) float32 dask.array<chunksize=(15, 199536), meta=np.ndarray>
    VelocityBeam2              (VelocityRange, time) float32 dask.array<chunksize=(15, 199536), meta=np.ndarray>
    VelocityBeam3              (VelocityRange, time) float32 dask.array<chunksize=(15, 199536), meta=np.ndarray>
    VelocityBeam4              (VelocityRange, time) float32 dask.array<chunksize=(15, 199536), meta=np.ndarray>
    CorrelationBeam1           (CorrelationRange, time) float32 dask.array<chunksize=(15, 199536), meta=np.ndarray>
    CorrelationBeam2           (CorrelationRange, time) float32 dask.array<chunksize=(15, 199536), meta=np.ndarray>
    ...                         ...
    EnsembleCount              (time) float64 dask.array<chunksize=(199536,), meta=np.ndarray>
    WaterTemperature           (time) float32 dask.array<chunksize=(199536,), meta=np.ndarray>
    Pressure                   (time) float64 dask.array<chunksize=(199536,), meta=np.ndarray>
    Heading                    (time) float32 dask.array<chunksize=(199536,), meta=np.ndarray>
    Pitch                      (time) float32 dask.array<chunksize=(199536,), meta=np.ndarray>
    Roll                       (time) float32 dask.array<chunksize=(199536,), meta=np.ndarray>
Attributes: (12/348)
    exportRevision:                               4489
    File_ins_runTime:                             51840125.0
    File_ins_runTime_description:                 Length of time data was col...
    File_ins_name:                                AD2CPRaw
    File_ins_name_description:                    Name of instrument
    File_ins_typeName:                            
    ...                                           ...
    instrumentOffsetTime:                         0.0
    instrumentOffsetTime_description:             Offset from instrument time...
    timeZoneOffset:                               -18000
    timeZoneOffset_description:                   Offset of platform local ti...
    instrumentOffsetTimeMatlab:                   0.0
    instrumentOffsetTimeMatlab_description:       Offset from instrument time...

## Steps 3-10: Big loop to process velocity data and save output

In [4]:
for x in range(0,len(start_times)):

    ## Subset glider df to one segment
    subsetgdf = gdf[(gdf.time >= start_times[x]) & (gdf.time <= end_times[x])]
    
    ind         = np.argwhere(np.isnan(subsetgdf.m_water_vx).ravel()==False).flatten()
    
    if len(ind)>0:
        ## Pull out last non-NaN lat/lon
        # ind1         = np.argwhere(np.isnan(subsetgdf.longitude).ravel()==False).flatten()
#         vx          = subsetgdf.m_water_vx.iloc[ind1[-1]]
#         vy          = subsetgdf.m_water_vy.iloc[ind1[-1]]
#         vx_start_lon = subsetgdf.longitude.iloc[ind1[0]]
#         vx_start_lat = subsetgdf.latitude.iloc[ind1[0]]
#         vx_end_lon   = subsetgdf.longitude.iloc[ind1[-1]]
#         vx_end_lat   = subsetgdf.latitude.iloc[ind1[-1]]
        
        
        vx          = subsetgdf.m_water_vx.iloc[ind[-1]]
        vy          = subsetgdf.m_water_vy.iloc[ind[-1]]
        vx_start_lon = subsetgdf.longitude.iloc[ind[0]]
        vx_start_lat = subsetgdf.latitude.iloc[ind[0]]
        vx_end_lon   = subsetgdf.longitude.iloc[ind[-1]]
        vx_end_lat   = subsetgdf.latitude.iloc[ind[-1]]
        vx_start_tm = subsetgdf.time.iloc[0]
        vx_end_tm   = subsetgdf.time.iloc[-1]
        
    if subsetgdf.depth.max() < 10:
            continue
            
            
    ad2cp_time_ind = np.where((tot_ad2cp.time.values >= start_times2[x]) & (tot_ad2cp.time.values <= end_times2[x]))[0]
    if len(ad2cp_time_ind) > 0:

        subset_ad2cp = tot_ad2cp.sel(time= slice(tot_ad2cp.time.values[ad2cp_time_ind[0]],tot_ad2cp.time.values[ad2cp_time_ind[-1]]))

        # Just check if there is still data after the subsetting 
        if len(subset_ad2cp.time) > 0:
            
            # Pre -process data before inversion
            subset_ad2cp = correct_sound_speed(subset_ad2cp)
            subset_ad2cp = qaqc_pre_coord_transform(subset_ad2cp, corr_threshold = 50, max_amplitude = 75)
            subset_ad2cp = beam_true_depth(subset_ad2cp)
            subset_ad2cp = binmap_adcp(subset_ad2cp)

            if 'AHRSRotationMatrix' in tot_ad2cp.variables:
                print('AD2CP has AHRS Unit')
                subset_ad2cp = beam2enu(subset_ad2cp)

            else:
                print('AD2CP has Standard Unit')
                subset_ad2cp.attrs['beam2xyz'] = subset_ad2cp.burst_beam2xyz.reshape(4,4)
                RotMatrix=calcAHRS_NI(subset_ad2cp['Heading'].values,subset_ad2cp['Roll'].values,subset_ad2cp['Pitch'].values)
                # Get your conditions
                subset_ad2cp['AHRSRotationMatrix'] = (('x','time'), RotMatrix)
                subset_ad2cp = beam2enu_NI(subset_ad2cp)

            subset_ad2cp = qaqc_post_coord_transform(subset_ad2cp, high_velocity_threshold=0.75, surface_depth_to_filter = 5)
           
            # ## Now ready for inversion!
            dz=10
            # O_ls, G_ls, bin_new,obs_per_bin  = inversion_NI(U=subset_ad2cp.UVelocity.values,V=subset_ad2cp.VVelocity.values,dz=dz,u_daverage=vx,v_daverage=vy,bins = subset_ad2cp['VelocityRange'].values, depth = subset_ad2cp['Pressure'].values,wDAC=5, wSmoothness=1)
            
            wDAC = 5
            wSmoothness = 1
            O_ls, G_ls, bin_new, obs_per_bin = inversion_NI(subset_ad2cp.UVelocity.values,subset_ad2cp.VVelocity.values,dz,vx,vy,subset_ad2cp['VelocityRange'].values,subset_ad2cp['Pressure'].values, wDAC, wSmoothness)
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Inversion", x ,"out of",len(start_times),"at" ,now)

            ###############################################
            #             Save master dataset             #
            ###############################################
            fname = "../Passengers/AD2CP_data_leg2/ad2cp_csv/RU36_2023_AD2CP_Processed_Segment_"+str(x)+".csv".format(dz,x)

            ## Make into a dataframe to save as a CSV
            d = {'inversion_u': np.real(O_ls), 'inversion_v': np.imag(O_ls), "inversion_depth": bin_new,
                 "start_lon": np.tile(vx_start_lon,len(bin_new)), "start_lat": np.tile(vx_start_lat,len(bin_new)),
                 "end_lon": np.tile(vx_end_lon,len(bin_new)), "end_lat": np.tile(vx_end_lat,len(bin_new)),
                 "start_tm": np.tile(vx_start_tm, len(bin_new)), "end_tm": np.tile(vx_end_tm, len(bin_new)),
                 "obs_per_bin": obs_per_bin}


            df = pd.DataFrame(data=d)
            df.to_csv(fname,index=False) 
            now = datetime.now().strftime("%m/%d/%y %H:%M:%S")
            print("Finished Writing Data", x ,"out of",len(start_times),"at" ,now)
            del subset_ad2cp
            
        else:
            del subset_ad2cp
            

AD2CP has Standard Unit
(9, 3208)
Finished Inversion 0 out of 241 at 01/23/24 14:29:43
Finished Writing Data 0 out of 241 at 01/23/24 14:29:43
AD2CP has Standard Unit
(9, 9612)
Finished Inversion 1 out of 241 at 01/23/24 14:30:22
Finished Writing Data 1 out of 241 at 01/23/24 14:30:22
AD2CP has Standard Unit
(9, 18480)
Finished Inversion 3 out of 241 at 01/23/24 14:31:38
Finished Writing Data 3 out of 241 at 01/23/24 14:31:38
AD2CP has Standard Unit
(9, 24730)
Finished Inversion 5 out of 241 at 01/23/24 14:33:21
Finished Writing Data 5 out of 241 at 01/23/24 14:33:21
AD2CP has Standard Unit
(9, 32170)
Finished Inversion 7 out of 241 at 01/23/24 14:35:40
Finished Writing Data 7 out of 241 at 01/23/24 14:35:40
AD2CP has Standard Unit
(9, 32167)
Finished Inversion 9 out of 241 at 01/23/24 14:37:54
Finished Writing Data 9 out of 241 at 01/23/24 14:37:54
AD2CP has Standard Unit
(9, 40320)
Finished Inversion 10 out of 241 at 01/23/24 14:40:56
Finished Writing Data 10 out of 241 at 01/23/24 1

## Load in Processed AD2CP Data and Export NetCDF

In [7]:
path ='../Passengers/AD2CP_data_leg2/ad2cp_csv/'
fname='../Passengers/AD2CP_data_leg2/processed_nc/Passengers2_ad2cp_20230525_to_20230614.nc'
files = np.sort(glob.glob(path+'*.csv'))
df = pd.concat(map(pd.read_csv, files))

inversion_dz = np.diff(df.inversion_depth)[0].astype(int)
inversion_depth         = np.arange(np.min(df.inversion_depth),np.max(df.inversion_depth)+inversion_dz,inversion_dz)
inversion_time          = np.empty(len(files))
inversion_time[:]       = np.NaN
inversion_time          = inversion_time.astype(pd.Timestamp)
inversion_start_time    = np.empty(len(files))
inversion_start_time[:] = np.NaN
inversion_start_time    = inversion_time.astype(pd.Timestamp)
inversion_end_time      = np.empty(len(files))
inversion_end_time[:]   = np.NaN
inversion_end_time      = inversion_time.astype(pd.Timestamp)
inversion_lat           = np.empty(len(files))
inversion_lat[:]        = np.NaN
inversion_lon           = np.empty(len(files))
inversion_lon[:]        = np.NaN
inversion_start_lat     = np.empty(len(files))
inversion_start_lat[:]  = np.NaN
inversion_start_lon     = np.empty(len(files))
inversion_start_lon[:]  = np.NaN
inversion_end_lat       = np.empty(len(files))
inversion_end_lat[:]    = np.NaN
inversion_end_lon       = np.empty(len(files))
inversion_end_lon[:]    = np.NaN
u_grid = np.empty((len(inversion_depth),len(files)))
u_grid[:] = np.NaN
v_grid = np.empty((len(inversion_depth),len(files)))
v_grid[:] = np.NaN


## Loop through by file, load in each file
for x in np.arange(0,len(files)):
    
    df = pd.read_csv(files[x])
    
    u_grid[np.arange(0,len(df.inversion_u.values)),x] = df.inversion_u.values
    v_grid[np.arange(0,len(df.inversion_v.values)),x] = df.inversion_v.values
    
    inversion_start_time[x] = pd.to_datetime(df.start_tm[0]).tz_localize(None)
    inversion_end_time[x] = pd.to_datetime(df.end_tm[0]).tz_localize(None)
    mid_time = inversion_end_time[x]-inversion_start_time[x]
    
    inversion_time[x] = inversion_start_time[x]+mid_time

    inversion_start_lat[x] = df.start_lat[0]
    inversion_start_lon[x] = df.start_lon[0]
    inversion_end_lat[x]   = df.end_lat[0]
    inversion_end_lon[x]   = df.end_lon[0]
    ## Lat/lon mid point
    inversion_lat[x]   = (df.start_lat[0]+df.end_lat[0])/2
    inversion_lon[x]   = (df.start_lon[0]+df.end_lon[0])/2


    
## Now stuff into an organized xarray dataset    
ds = xr.Dataset(
    data_vars=dict(
        u_grid     = (["depth", "time"], u_grid),
        v_grid     = (["depth", "time"], v_grid),
        latitude   = (["time"], inversion_lat),
        longitude  = (["time"], inversion_lon),
        start_lat  = (["time"], inversion_start_lat),
        start_lon  = (["time"], inversion_start_lon),
        end_lat    = (["time"], inversion_end_lat),
        end_lon    = (["time"], inversion_end_lon),
        start_time = (["time"], inversion_start_time),
        end_time   = (["time"], inversion_end_time)
    ),
    coords=dict(
        time  = inversion_time,
        depth = inversion_depth
    ),
    attrs=dict(description="PASSENGERS 2: Nortek AD2CP least-squares inversion velocity profile dataset"),
)

## Sort by time because files may not have in read in chronological order
ds = ds.sortby(ds.time)
#save output to netcdf
ds.to_netcdf(fname)
ds

<xarray.Dataset>
Dimensions:     (depth: 98, time: 97)
Coordinates:
  * time        (time) datetime64[ns] 2023-05-25T18:02:43.770000 ... 2023-06-...
  * depth       (depth) float64 5.0 15.0 25.0 35.0 ... 945.0 955.0 965.0 975.0
Data variables:
    u_grid      (depth, time) float64 0.5381 0.5343 0.5324 ... nan nan nan
    v_grid      (depth, time) float64 0.0731 0.003893 -0.07406 ... nan nan nan
    latitude    (time) float64 38.03 38.03 38.04 38.04 ... 37.77 37.74 37.71
    longitude   (time) float64 -63.45 -63.44 -63.43 ... -62.78 -62.8 -62.83
    start_lat   (time) float64 38.03 38.03 38.03 38.04 ... 37.78 37.75 37.72
    start_lon   (time) float64 -63.45 -63.45 -63.43 ... -62.77 -62.79 -62.81
    end_lat     (time) float64 38.03 38.03 38.04 38.04 ... 37.75 37.72 37.71
    end_lon     (time) float64 -63.45 -63.43 -63.42 ... -62.79 -62.81 -62.84
    start_time  (time) datetime64[ns] 2023-05-25T17:48:02.520000 ... 2023-06-...
    end_time    (time) datetime64[ns] 2023-05-25T18:02:43.770000 ... 2023-06-...
Attributes:
    description:  PASSENGERS 2: Nortek AD2CP least-squares inversion velocity...